In [5]:
import sys  
!{sys.executable} -m pip install pandas

  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 26.0 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [7]:
import requests 
from bs4 import BeautifulSoup
import random
import pandas as pd

In [8]:
title = "Product Manager"
location = "United States"

In [9]:
list_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Product%2BManager&location=United%2BStates&start=120"

response = requests.get(list_url)

list_data = response.text
list_soup = BeautifulSoup(list_data, "html.parser")
page_jobs = list_soup.find_all("li")

In [10]:
id_list = []

In [11]:

for job in page_jobs:
    base_card_div = job.find("div", {"class": "base-card"})
    job_id = base_card_div.get("data-entity-urn").split(":")[3]
    print(job_id)
    id_list.append(job_id)

3944668599
3945632970
3937289445
3933134640
3940865810
3947117951
3941805244
3940870697
3943485443
3934732794


In [12]:
id_list

['3944668599',
 '3945632970',
 '3937289445',
 '3933134640',
 '3940865810',
 '3947117951',
 '3941805244',
 '3940870697',
 '3943485443',
 '3934732794']

In [13]:
import time
import requests
from bs4 import BeautifulSoup

job_list = []

for job_id in id_list:
    job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
    job_response = requests.get(job_url)

    if job_response.status_code == 429:
        # If the status code is 429 (rate limit exceeded), wait for the specified time
        retry_after = int(job_response.headers.get("Retry-After", 10))  # Default to 10 seconds if header not present
        print(f"Rate limit exceeded. Waiting for {retry_after} seconds before retrying...")
        time.sleep(retry_after)
        
        # Make the request again
        job_response = requests.get(job_url)

    print(job_response.status_code)
    job_soup = BeautifulSoup(job_response.text, "html.parser")
    job_post = {}
    try:
        job_post["job_title"] = job_soup.find("h2", {"class": "top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"}).text.strip()
    except:
        job_post["job_title"] = None
    try:
        job_post["company_name"] = job_soup.find("a", {"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
    except:
        job_post["company_name"] = None
    try:
        job_post["time_posted"] = job_soup.find("span", {"class": "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
    except:
        job_post["time_posted"] = None
    try:
        job_post["num_applicants"] = job_soup.find("span", {"class": "num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"}).text.strip()
    except:
        job_post["num_applicants"] = None
    
    job_list.append(job_post)

200
200
200
200
200
200
200
200
200
200


In [14]:
job_list

[{'job_title': 'Lead Product Manager, Payments',
  'company_name': 'Gemini',
  'time_posted': '4 days ago',
  'num_applicants': None},
 {'job_title': 'Senior Manager, Product Technical Services',
  'company_name': 'LinkedIn',
  'time_posted': '4 days ago',
  'num_applicants': '58 applicants'},
 {'job_title': 'Product Manager (US remote, EST only)',
  'company_name': 'Digimarc',
  'time_posted': '1 week ago',
  'num_applicants': '114 applicants'},
 {'job_title': 'Product Manager',
  'company_name': 'Mach7 Technologies',
  'time_posted': '2 weeks ago',
  'num_applicants': None},
 {'job_title': 'Platform Product Manager, Standards',
  'company_name': 'Airbnb',
  'time_posted': '4 days ago',
  'num_applicants': None},
 {'job_title': 'Product Manager',
  'company_name': 'Motion Recruitment',
  'time_posted': '1 day ago',
  'num_applicants': '75 applicants'},
 {'job_title': 'Product Manager',
  'company_name': 'Oracle',
  'time_posted': '3 days ago',
  'num_applicants': '139 applicants'},
 {

In [15]:
jobs_df = pd.DataFrame(job_list)
jobs_df

,job_title,company_name,time_posted,num_applicants
0,"Lead Product Manager, Payments",Gemini,4 days ago,None
1,"Senior Manager, Product Technical Services",LinkedIn,4 days ago,58 applicants
2,"Product Manager (US remote, EST only)",Digimarc,1 week ago,114 applicants
3,Product Manager,Mach7 Technologies,2 weeks ago,None
4,"Platform Product Manager, Standards",Airbnb,4 days ago,None
5,Product Manager,Motion Recruitment,1 day ago,75 applicants
6,Product Manager,Oracle,3 days ago,139 applicants
7,Product Manager,Microsoft,4 days ago,93 applicants
8,Product Manager,Steneral Consulting,5 days ago,None
9,Product Manager,CONMED Corporation,2 weeks ago,None


In [16]:
jobs_df.to_csv('United_States_Product_Manager.csv', index = False)
